In [ ]:
import os
from sqlalchemy import create_engine, Column, Integer, String, Text, DateTime
from sqlalchemy.orm import declarative_base, sessionmaker
from dotenv import load_dotenv
from langchain_cohere import ChatCohere
from langchain.agents import AgentExecutor, create_react_agent
from langchain.tools import Tool
from langchain import hub
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain.schema import StrOutputParser
from uuid import uuid4
from datetime import datetime
import pandas as pd

# Load environment variables from .env file
load_dotenv()

# Set up SQLite database
DATABASE_URL = "sqlite:///C:/Users/jarmo/gits/interface_app_once_a_day/backend/chat_history.db"
Base = declarative_base()
engine = create_engine(DATABASE_URL)
SessionLocal = sessionmaker(autocommit=False, autoflush=False, bind=engine)

# Define ChatMessage model
class ChatMessage(Base):
    __tablename__ = "chat_messages"
    id = Column(Integer, primary_key=True, index=True)
    session_id = Column(String, index=True)
    role = Column(String)
    content = Column(Text)
    timestamp = Column(DateTime, default=datetime.utcnow)

# Create the database tables
Base.metadata.create_all(bind=engine)

# Function to get a new database session
def get_db():
    db = SessionLocal()
    try:
        yield db
    finally:
        db.close()

# Function to save chat message to the database
def save_chat_message(session_id, role, content, db):
    db_message = ChatMessage(session_id=session_id, role=role, content=content)
    db.add(db_message)
    db.commit()
    db.refresh(db_message)
    return db_message

# Function to get chat message history from the database
def get_chat_message_history(session_id, db):
    return db.query(ChatMessage).filter(ChatMessage.session_id == session_id).all()

# Custom ChatMessageHistory class to manage chat history
class ChatMessageHistory:
    def __init__(self, messages=None):
        if messages is None:
            messages = []
        self.messages = messages

    def add_messages(self, new_messages):
        self.messages.extend(new_messages)

    def get_messages(self):
        return self.messages

# Initialize Cohere client with API key from environment variables
llm = ChatCohere(
    cohere_api_key=os.getenv("COHERE_API_KEY")
)


general_chat_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are an expert of general chat. You will answer any question with a short answer in an easy to read structured form.",
        ),
        ("human", "{input}"),
    ]
)

general_chat = general_chat_prompt | llm | StrOutputParser()

# List of tools for the agent to use
tools_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are an expert of tools. You will list all of the current tools possible to use.",
        ),
        ("human", "{input}"),
    ]
)

tools_chat = tools_prompt | llm | StrOutputParser()

# Define the interface prompt template for LLM interface app suggestions
interface_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are an llm interface app expert. You find produce suggestions for llm interface applications that is easy to make money with.",
        ),
        ("human", "{input}"),
    ]
)

# Create the interface chat chain
interface_chat = interface_prompt | llm | StrOutputParser()

# Define the cooking prompt template for easy recipes
cooking_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are an expert of cooking easy to make food. You give easy to make recipes for people who are busy and don't have time to cook.",
        ),
        ("human", "{input}"),
    ]
)

# Create the cooking chat chain
cooking_chat = cooking_prompt | llm | StrOutputParser()

new_tool_suggestion = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            """
            You are an expert of suggesting new tools to my app. You will suggest a new tool to use in my agent. Also dont suggest ones that i already use. 
         My code is a Jupyter Notebook that sets up a chat application using LangChain and Cohere, with SQLite for storing chat message histories. Here's a brief explanation of the key components:

Environment Setup:

Loads environment variables from a .env file.
Sets up an SQLite database using SQLAlchemy.
Database Models and Functions:

Defines a ChatMessage model to store chat messages.
Includes functions to save and retrieve chat messages from the database.
Custom ChatMessageHistory Class:

Manages chat message histories, allowing messages to be added and retrieved.
Cohere Client Initialization:

Initializes the Cohere client using an API key from environment variables.
Prompt Templates and Chat Chains:

Defines various prompt templates for different types of chats (general chat, interface app suggestions, cooking recipes).
Creates chat chains using these prompt templates and the Cohere client.
Tools Definition:

Defines tools for the agent to use, each associated with a specific chat chain.
Agent Setup:

Pulls an agent prompt from the hub and creates an agent using the defined tools.
Creates an executor for the agent.
Chat Agent with Message History:

Defines a function to get chat message history for a session.
Creates a chat agent that uses this message history.
Main Loop:

Generates a unique session ID.
Handles user input, saves chat messages to the database, and generates responses using the chat agent.
This setup allows the chat application to remember previous messages and provide context-aware responses based on the stored chat history.
        """,
        ),
        ("human", "{input}"),
    ]
)

new_tool_chat = new_tool_suggestion | llm | StrOutputParser()


# Define tools for the agent to use
tools = [
    Tool.from_function(
        name="General Chat",
        description="For when you need to chat about anything. The question will be a string. Return short answer in an easy to read structured form.",
        func=general_chat.invoke,
    ),
    Tool.from_function(
        name="Interface Chat",
        description="For when you need to chat about interface apps. The question will be a string. Return in an easy to read structured form.",
        func=interface_chat.invoke,
    ),
    Tool.from_function(
        name="Cooking Chat",
        description="For when you need to chat about cooking. The question will be a string. Return in an easy to read structured form.",
        func=cooking_chat.invoke,
    ),
    Tool.from_function(
        name="Get Tools",
        description="Only list all of the current tools possible to use and nothing else. Return list of tools in bold and their descriptions in csv format.",
        func=tools_chat.invoke,
    ),
    Tool.from_function(
        name="New Tool Suggestor",
        description="Suggest a new tool to add to the list of tools. The question will be a string. Return in an easy to read structured form.",
        func=new_tool_chat.invoke,
    ),

    Tool.from_function(
        name="CSV Analyzer",
        description="Analyze a CSV file and return summary statistics. The input will be a file path to the CSV file.",
        func=lambda file_path: pd.read_csv(file_path).describe().to_dict(),
    ),
     
]

# Pull agent prompt from hub
agent_prompt = hub.pull("hwchase17/react-chat")
# Create the agent using the pulled prompt and tools
agent = create_react_agent(llm, tools, agent_prompt)
# Create an executor for the agent
agent_executor = AgentExecutor(agent=agent, tools=tools)

# Function to get chat message history for a session
def get_memory(session_id):
    db = next(get_db())
    messages = get_chat_message_history(session_id=session_id, db=db)
    # Convert database messages to a format suitable for ChatMessageHistory
    formatted_messages = [{"role": msg.role, "content": msg.content} for msg in messages]
    return ChatMessageHistory(formatted_messages)

# Create chat agent with message history
chat_agent = RunnableWithMessageHistory(
    agent_executor,
    get_memory,
    input_messages_key="input",
    history_messages_key="chat_history",

)

# Generate a unique session ID
SESSION_ID = str(uuid4())
print(f"Session ID: {SESSION_ID}")

# Main loop to handle user input and generate responses
while (q := input("> ")) != "exit":
    db = next(get_db())
    save_chat_message(SESSION_ID, "human", q, db)
    
    response = chat_agent.invoke(
        {
            "input": q
        },
        {"configurable": {"session_id": SESSION_ID, "message_history": get_memory(SESSION_ID)}},
    )
    
    save_chat_message(SESSION_ID, "system", response["output"], db)
    print(response["output"])

c:\Users\jarmo\anaconda3\envs\learn_langchain-env\lib\site-packages\langsmith\client.py:241: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


Session ID: 6bd3db7d-4150-452e-8715-53f57996f5e3
I've provided a comprehensive list of tools across various categories, from hand tools to technology and crafting. This list is a general overview, and I can offer more specific details if you're interested in a particular type of tool or category. Just let me know if you'd like me to dive deeper into any of these areas!
I've provided a list of various tools in different categories, including hand tools, power tools, gardening tools, kitchen tools, automotive tools, and technology/software tools. This list is just a general overview, and there are many more tools available depending on the specific field or industry. If you'd like to explore a particular category in more detail, feel free to ask!
That's an impressive list of tools and capabilities! It's fascinating to see how AI language models like yourself can process and generate text, learn from data, and maintain context in conversations. The ability to adapt responses to user prefe